In [ ]:
import tifffile
import numpy as np
import statistics
import os
from tqdm import tqdm
from glob import glob
from scipy import ndimage as ndi
from matplotlib import pyplot as plt
from skimage.io import imsave
from skimage import (
    measure,
    morphology,
    exposure,
    filters,
    img_as_ubyte,
    feature,
)
from commons import crop_image_box, find_center_mask
from skimage import feature
from sklearn.cluster import KMeans

In [ ]:
def plot_images(images, color='gray', names=[]):
    """
        Função para plotar array de imagens, essa função não é perfeita
        mas serve bem...
    """

    if len(names) == 0:
        names = [""] * len(images)

    if len(images) == 1:
        plt.figure(figsize=(10, 8))
        plt.imshow(images[0], color)

        return plt.show()

    fig, ax = plt.subplots(1,
                           len(images),
                           figsize=(15, 20))

    for index, arr in enumerate(images):
        ax[index].imshow(arr, cmap=color)
        ax[index].set_title(names[index])

    plt.show()
    
def read_image_frames_seq(path):
    def parser_image_name(image_name):

        *_, name = image_name.split("/")
        name, *_ = name.split(".")

        return int(name)
    
    arr = []
    
    for index in glob(path + "/*"):
        try:
            image_name = parser_image_name(index)
            
            arr.append(image_name)
            
        except:
            continue
    
    image_path_sorted = sorted(arr)

    image_unique_name = lambda x: f"{path}/{x}.tif"

    return list(map(image_unique_name, image_path_sorted))


In [ ]:
def build_volume_from_directory(arr_paths, with_rgb=False):
    """
        Ler todas as imagens do diretório e cria um bloco de imagens
    """
    if with_rgb:
        return np.asarray([tifffile.imread(img) for img in arr_paths])

    return np.asarray([tifffile.imread(img)[:, :, 0] for img in arr_paths])

In [ ]:
def find_bighest_cluster_area(clusters):
    """
        Essa função deve receber uma imagem segmentada (Clusters)
        Retorna a área do maior cluster
    """
    regions = measure.regionprops(clusters)

    def area(item): return item.area

    return max(map(area, regions))


def find_best_larger_cluster(image_mask):

    clusters = image_mask.copy()

    if statistics.mode(clusters.flatten()):
        clusters = np.invert(clusters)

    clusters_labels = measure.label(clusters, background=0)

    cluster_size = find_bighest_cluster_area(clusters_labels)

    return morphology.remove_small_objects(
        clusters,
        min_size=(cluster_size-1),
        connectivity=8
    )

In [ ]:
def binarize_image(arr):
    return arr > filters.threshold_triangle(arr)

In [ ]:
def find_broiler_roi(frame, background):
    def second_tecnique(frame):

        binary_frame = binarize_image(exposure.equalize_hist(frame))

        best_cluster = find_best_larger_cluster(binary_frame)

        merged = binarize_image(
            binary_frame.astype("uint8") + best_cluster.astype("uint8")
        )

        best_cluster = find_best_larger_cluster(merged)

        return binarize_image(best_cluster)

    def first_tecnique(frame, background):

        mask_bin = binarize_image(
            np.subtract(
                exposure.equalize_hist(background),
                exposure.equalize_hist(frame),
            )
        )

        best_cluster = find_best_larger_cluster(mask_bin)

        return binarize_image(best_cluster)

    mask = (second_tecnique(frame) +
            first_tecnique(frame, background)).astype(dtype=bool)

    if statistics.mode(mask.flatten()):
        return np.invert(mask)

    # Arremata

    closed = morphology.closing(mask, morphology.disk(5))

    fill = ndi.binary_fill_holes(closed)

    final = find_best_larger_cluster(fill)

    return final.astype(dtype=bool)


In [ ]:
pixel_size = 100

for path in glob("base/images/*")[:1]:

    seq_frames = read_image_frames_seq(path)
    seq_frames_volume = build_volume_from_directory(seq_frames)
    background = np.median(seq_frames_volume, axis=0).astype(dtype=np.uint8)

    image_name = path.split('/')[2]

    try:
        os.makedirs(f'tests/{image_name}')
    except:
        pass

    for index, frame in enumerate(tqdm(seq_frames_volume)):

        mask = find_broiler_roi(frame, background)

        try:
            crop_frame = crop_image_box(image=frame,
                                        shape=find_center_mask(mask),
                                        margin_pixel=pixel_size)

            if not crop_frame.size:
                continue
            if crop_frame.shape != (pixel_size * 2, pixel_size * 2):
                continue

        except Exception as e:
            continue

        imsave(f"tests/{image_name}/{index}.tif", crop_frame)
